In [2]:
import numpy as np

In [17]:
def LUdecomp(a):
    n = len(a)
    for k in range(0, n-1):
        if a[k, k] == 0:
            raise ValueError("Descompunerea LU eșuează: pivot nul.")
        for i in range(k+1, n):
            if a[i, k] != 0.0:
                lam = a[i, k] / a[k, k]
                a[i, k+1:n] = a[i, k+1:n] - lam * a[k, k+1:n]
                a[i, k] = lam
    return a

In [20]:
n = 3

#matrix =  np.random.rand(n,n)



A = np.array([
    [4, 2, 3],
    [2, 7, 5.5],
    [6, 3, 12.5]
])

valid = True

dU = [2, 3, 4]

for k in range(1,n+1):
    Ak = A[:k, :k]
    det = np.linalg.det(Ak)

    if det == 0:
        valid = False
        print("Nu se poate calcula o descompunere LU")

        break
 
if valid:
    LU = LUdecomp(A.copy())
    print(LU)

[[4.  2.  3. ]
 [0.5 6.  4. ]
 [1.5 0.  8. ]]
